# dspy methodology 101

1. programming
   1. LMs (tasks)
   2. signatures (i/o - eg `"context: list[str], question: str -> answer: str"`) - compiling leads to better prompts than humans write
      1. tasks, instruct the model what it needs to do
      2. underlying dSPY compiler will do the optimization, rather than brittle prompts
   3. modules (ie `dspy.Predict`, `dspy.ChainOfThought`)
      1. prompting techniques
2. evaluation
3. optimization

## TOC:
* [intro](#dspy-methodology-101)
* [LMs](#set-a-generator-lm)
* [modules](#testing-with-dspy-modules)

## set a generator LM

<a class="anchor" id="LM"></a>

In [1]:
import dspy
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

if OPENAI_API_KEY:
    print("OPENAI API Key found!")
else:
    print("OPENAI API Key not found!")
if ANTHROPIC_API_KEY:
    print("ANTHROPIC API Key found!")
else:
    print("ANTHROPIC API Key not found!")

lm = dspy.LM('openai/gpt-4o-mini', temperature=0.9, max_tokens=3000, stop=None, cache=False, api_key=OPENAI_API_KEY)
dspy.configure(lm=lm)

OPENAI API Key found!
ANTHROPIC API Key found!


## testing with dspy modules

Each built-in module abstracts a prompting technique (like chain of thought or ReAct). Crucially, they are generalized to handle any signature.

Modules help you describe AI behavior as code, not strings.
<a class="anchor" id="MODULES"></a>

- `dspy.Predict`: Basic predictor. Does not modify the signature. Handles the key - `forms of learning (i.e., storing the instructions and demonstrations and - `updates to the LM).
- `dspy.ChainOfThought`: Teaches the LM to think step-by-step before committing to - `the signature's response.
- `dspy.ProgramOfThought`: Teaches the LM to output code, whose execution results - `will dictate the response.
- `dspy.ReAct`: An agent that can use tools to implement the given signature.
- `dspy.MultiChainComparison`: Can compare multiple outputs from ChainOfThought to produce a final prediction.

In [2]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

len(lm.history)
lm.history[0].keys()

5766.904


dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [ ]:
with dspy.context(lm=dspy.LM('openai/gpt-3.5-turbo')):
    math = dspy.ChainOfThought("question -> answer: float")
    result = math(question="Two dice are tossed. What is the probability that the sum equals two?")
    print("Result:", result)

print(f"\nHistory length: {len(lm.history)}")
lm.history[0].keys()

with dspy.context(lm=dspy.LM('anthropic/claude-3-7-sonnet-20250219', temperature=0.9, max_tokens=3000, stop=None, cache=False, api_key=ANTHROPIC_API_KEY)):
    math = dspy.ChainOfThought("question -> answer: float")
    result = math(question="Two dice are tossed. What is the probability that the sum equals two?")
    print('claude-3-opus Result:', result) # where result.answer would display the answer but no reasoning which comes from the dspy.ChainOfThought module

print(f"\nHistory length: {len(lm.history)}")
lm.history[-1].keys()

Result: Prediction(
    reasoning='The sum of two dice can range from 2 to 12. To get a sum of 2, only one combination is possible: (1, 1). There are a total of 36 possible outcomes when two dice are tossed (6 outcomes for the first die and 6 outcomes for the second die). Therefore, the probability of getting a sum of 2 is 1/36.',
    answer=0.027777777777777776
)

History length: 5
claude-3-opus Result: Prediction(
    reasoning='When tossing two dice, each die can show a face with 1, 2, 3, 4, 5, or 6 dots, with equal probability. So there are 6 possible outcomes for each die.\n\nThe total number of possible outcomes when tossing two dice is 6 × 6 = 36, and each of these outcomes is equally likely.\n\nFor the sum to equal 2, both dice must show a 1. This is the only way to get a sum of 2.\nSo there is only 1 favorable outcome: (1,1).\n\nThe probability is therefore:\nNumber of favorable outcomes / Total number of possible outcomes = 1/36 ≈ 0.0277777...',
    answer=0.02777777777777778

dict_keys(['prompt', 'messages', 'kwargs', 'response', 'outputs', 'usage', 'cost', 'timestamp', 'uuid', 'model', 'response_model', 'model_type'])

In [4]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal['positive', 'negative', 'neutral'] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.85
)

In [5]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning='The context provides information about David Gregory, mentioning that he inherited Kinnairdy Castle in 1664. Therefore, the name of the castle he inherited is Kinnairdy Castle.',
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'
)

In [6]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. Announces iPhone 14
['Introduction', 'CEO Announcement', 'Features Overview']
[{'name': 'Apple Inc.', 'type': 'Organization'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person'}]


In [8]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="World Cup 2002")

print(f"# {article.title}\n")
for section in article.sections:
    print(f"{section}\n")

# Overview of the 2002 FIFA World Cup

## Tournament Structure

### Host Nations
The 2002 FIFA World Cup was uniquely co-hosted by two nations, South Korea and Japan. This was the first time in history that the World Cup was held in Asia and also the first time it was jointly hosted by two different countries. This collaborative effort showcased both nations' capabilities and commitment to putting on a world-class tournament.

### Participating Teams
A total of 32 teams competed in the 2002 World Cup, representing various confederations from around the globe. This marked an expansion from the previous tournament's 24-team format, allowing for more countries to participate and showcase their talent on the world stage. The teams were divided into eight groups of four, with the top two from each group advancing to the knockout stage.

### Match Format
The tournament utilized a group-stage format followed by knockout rounds. Each team played the others in their group once, earning points f

--> compile these modules into optimized prompts and finetunes